# Training & Evaluation

## ##Training & Evaluation is currently being carried out in google Colaboratory due to less free resources available in IBM watson studio

# Download Data

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [5]:
download_file_cos(credentials_2,'train_feat_eng.csv','train_feat_eng.csv')
download_file_cos(credentials_2,'test_feat_eng.csv','test_feat_eng.csv')
download_file_cos(credentials_2,'sample_submission.csv','sample_submission.csv')
!ls

File Downloaded
File Downloaded
File Downloaded
sample_submission.csv  test_feat_eng.csv  train_feat_eng.csv


# Creating training, validation and testing data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
%matplotlib inline

In [ ]:
tf.__version__

In [8]:
train=pd.read_csv('train_feat_eng.csv')
train.head()

,pairs_enc,distance,bond_int,type_enc,scalar_coupling_constant
0,2,1.091953,1,0,84.8076
1,11,1.783120,1,3,-11.2570
2,11,1.783147,1,3,-11.2548
3,11,1.783157,1,3,-11.2543
4,2,1.091952,1,0,84.8074


In [9]:
train.shape

(4658147, 5)

In [10]:
test=pd.read_csv('test_feat_eng.csv')
test.head()

,pairs_enc,distance,bond_int,type_enc
0,2,2.261178,0,2
1,2,1.062099,1,0
2,11,3.323277,0,6
3,2,1.062099,1,0
4,2,2.261178,0,2


In [11]:
test.shape

(2505542, 4)

In [12]:
train.columns

Index(['pairs_enc', 'distance', 'bond_int', 'type_enc',
       'scalar_coupling_constant'],
      dtype='object')

In [13]:
x=train.drop('scalar_coupling_constant',axis=1)
x.head()

,pairs_enc,distance,bond_int,type_enc
0,2,1.091953,1,0
1,11,1.783120,1,3
2,11,1.783147,1,3
3,11,1.783157,1,3
4,2,1.091952,1,0


In [14]:
y=train['scalar_coupling_constant']
y.head()

0    84.8076
1   -11.2570
2   -11.2548
3   -11.2543
4    84.8074
Name: scalar_coupling_constant, dtype: float64

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3)

In [17]:
print('x_train: {}   y_train: {}'.format(x_train.shape,y_train.shape))
print('x_val: {}   y_val: {}'.format(x_val.shape,y_val.shape))

x_train: (3260702, 4)   y_train: (3260702,)
x_val: (1397445, 4)   y_val: (1397445,)


In [18]:
x_test=test
x_test.shape

(2505542, 4)

# Training & Evaluation

In [20]:
input_shape=(4,)
model_nn=keras.Sequential()
model_nn.add(keras.layers.Dense(256,activation='relu',input_shape=input_shape))
model_nn.add(keras.layers.Dense(128,activation='relu'))
model_nn.add(keras.layers.Dense(64,activation='relu'))
model_nn.add(keras.layers.Dense(8,activation='relu'))
model_nn.add(keras.layers.Dense(1))

model_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               1280      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 42,961
Trainable params: 42,961
Non-trainable params: 0
_________________________________________________________________


In [21]:
model_nn.compile(optimizer='adam',loss='mean_absolute_error')

In [ ]:
model_nn.fit(x_train,y_train,epochs=5,batch_size=512,validation_data=(x_val,y_val))

# Serialize the model

In [ ]:
tf.keras.models.save_model(model_nn,'kaggle_model_nn_03.h5',overwrite=True,include_optimizer=True)

In [ ]:
!ls

# Create Submission file

In [ ]:
test_predictions=model.predict(x_test,verbose=1,batch_size=10000)
test_predictions[:10]

In [ ]:
test_predictions.shape

In [ ]:
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['scalar_coupling_constant']=test_predictions
sample_submission.head()

In [ ]:
sample_submission.to_csv('sample_submission_test_3.csv',index=False)

In [ ]:
!ls

In [ ]:
!head sample_submission_test_3.csv

# Upload to IBM object store

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [ ]:
upload_file_cos(credentials_2,'sample_submission_test_3.csv','sample_submission_test_3.csv')

In [ ]:
upload_file_cos(credentials_2,'kaggle_model_nn_03.h5','kaggle_model_nn_03.h5')